In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
from os import path

from dff.webdrivers import init_driver
from dff.helpers import join_or_make, get_args, get_fb_id_from_url
from dff.helpers import construct_social_graph, save_photos, save_friends_data
from dff.scrapper import login, get_friends_section_url, make_friends_data
from dff.scrapper import get_user_about_section_info, get_user_photos, get_friends
from dff.scrapper import make_mutual_friends
from dff.scrappers.about import AboutPage
from dff.scrappers.login import Login
from dff.scrappers.home import HomePage
from dff.scrappers.friends import Friends

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
from os import path

from dff.webdrivers import init_driver
from dff.helpers import get_args
from dff.scrappers.about import AboutPage
from dff.scrappers.login import Login
from dff.scrappers.home import HomePage
from dff.scrappers.friends import Friends
from dff.scrappers.photos import Photos

In [2]:
conf = get_args('config.yml')

print('init driver')
driver = init_driver(conf['driver']['name'], conf['driver']['maximized'], 
    conf['driver']['headless'], conf['driver']['surpress_notifications'])

init driver


In [3]:
login = Login(driver, conf['auth']['fb_user'], conf['auth']['fb_pass'], 'config/scrappers/login.yml', 
              debug=conf['debug'])
login.start()
print login.data

{'logged_in': True}


In [4]:
screenshot_config = conf['screenshots']['home']
screenshot_config['path'] = path.join(conf['user_folder'], conf['screenshots']['path'])
home_page = HomePage(driver, screenshot_config, 'config/scrappers/home.yml', debug=conf['debug'])
home_page.start()
print home_page.data

{'profile': {'url': u'https://www.facebook.com/athmane.oran', 'username': u'athmane.oran', 'name': u'Edit Athmane Oranais', 'id': u'1353966325'}}


In [ ]:
about_page = AboutPage(driver, home_page.data['profile']['username'], 'config/scrappers/about.yml', 
                      debug=conf['debug'])
about_page.start()
print about_page.data

In [ ]:
friends = Friends(driver, home_page.data['profile']['url'], 'config/scrappers/friends.yml', 
                  base_path=conf['user_folder'], debug=conf['debug'])
friends.start()
friends.save()
print friends.data['friends'][0]

In [6]:
photos = Photos(driver, home_page.data['profile']['url'], 'config/scrappers/photos.yml', 
                  base_path=conf['user_folder'], debug=conf['debug'])
photos.start()
photos.save()
print photos.data['photos']

[{'img_url': u'https://fb-s-c-a.akamaihd.net/h-ak-fbx/v/t1.0-0/p206x206/12803098_10208834453362671_6902062495427226020_n.jpg?oh=324f7458c83cb9d0e74d1daf8cf5f4fc&oe=5ACB4E3A&__gda__=1522729294_7ce1547a406288ae4963573048e5df11', 'id': u'pic_10208834453362671', 'img_path': u'data/athmane.oran/photos/pic_10208834453362671.jpg'}, {'img_url': u'https://fb-s-b-a.akamaihd.net/h-ak-fbx/v/t1.0-0/p206x206/12472533_10208834442882409_2315754954566649144_n.jpg?oh=231ee884a094dc96e50ad6d0c4cfd078&oe=5A8BAD1F&__gda__=1523769451_45cc890bd07fa9b3f0f8237ba08de1a2', 'id': u'pic_10208834442882409', 'img_path': u'data/athmane.oran/photos/pic_10208834442882409.jpg'}, {'img_url': u'https://fb-s-c-a.akamaihd.net/h-ak-fbx/v/t1.0-0/p206x206/522456_10203103321887966_258849789_n.jpg?oh=a4add6c9a05589142549d86a16c0fa70&oe=5AD00CA7&__gda__=1519523630_4ff0a4667a37077ff164cfc01f9abbcd', 'id': u'pic_10203103321887966', 'img_path': u'data/athmane.oran/photos/pic_10203103321887966.jpg'}, {'img_url': u'https://fb-s-a-a.ak

In [ ]:
conf = get_args('config.yml')

print('init driver')
# TODO: make all args accessible via dict notation
driver = init_driver(conf['driver']['name'], conf['driver']['maximized'], 
    conf['driver']['headless'], conf['driver']['surpress_notifications'])

print('logging to FB account')
logged_in = login(driver, conf['base_url'], conf['auth']['fb_user'],
    conf['auth']['fb_pass'], conf['user_folder'])

if(logged_in):
    friends_section_url = get_friends_section_url(driver, conf['base_url'])
    print('Getting friends blocks')
    friends_blocks = get_friends(driver, friends_section_url)

    print('extracting data from friends blocks')
    friends_data = make_friends_data(friends_blocks)
    
    for i, friend_data in enumerate(friends_data):
        friends_data[i]['mutual_friends'] = make_mutual_friends(driver, friend_data['url'])

    print('Saving friends data')
    save_friends_data(friends_data, conf['user_folder'])

    print('Download friends photos')
    save_photos(friends_data, join_or_make(conf['user_folder'], 'friends_photos'), 
        suffix='.thumb')

    print('Getting About user info')
    print(get_user_about_section_info(driver))
    
    photos = get_user_photos(driver)

    print('Getting photos available on photos section')
    save_photos(photos, join_or_make(conf['user_folder'], 'photos'))
    
    print('constructing the social graph of the account')
    friends_graph = construct_social_graph(conf['auth']['fb_user'], friends_data)
    print(nx.info(friends_graph))
    
    plt.figure(3, figsize=(15, 10)) 
    plt.title('Graphe des amis')
    nx.draw(friends_graph, with_labels=True)
else:
    print "Couldn't login"

In [ ]:
d['b']